# Image Processing in Python

**Part of the IAFIG-RMS *Python for Bioimage Analysis* Course.**

*Dr Chas Nelson*

2019-12-09 1300--1430

## Aim

To carry out key image processing operations in Python.

## ILOs

* Appreciate the capabilities of `scikit-image` for image processing in a Python environment
* Apply known image processing techniques (e.g. smoothing) in a Python environment
* Recognise additional image processing techniques (e.g. deconvolution) that are possible in a Python environment
* Relate global grayscale thresholding and the logical array to segmentation and binary images
* Extract features of objects from segmented images

## Imports

In [ ]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import ipywidgets as widgets
import IPython.display as ipyd
from skimage import io

## Data

The image we will use for the rest of this tutorial is from the Broad Bioimage Benchmark Collection data set BBBC0034v1 (https://data.broadinstitute.org/bbbc/; Thirstrup et al. 2018).

See https://data.broadinstitute.org/bbbc/BBBC034/ for the full description; however, the key points are:

* $1024 \times 1024 \times 52$ pixels
* $65 \times 65 \times 290$ nm/pixel
* 4 channels (each stored as separate files):
  * Cell membrane label (C=0)
  * Actin label (C=1)
  * DNA label (C=2)
  * Brightfield image (C=3)
  
The below cell can be run to create a local link to the data that we downloaded in the previous session. You only need to run this cell once and then you may comment it out.

In [ ]:
# import os

# os.symlink('../../01_images-in-python/assets/bbbc034v1','./assets/bbbc034v1')

## Contrast and Histogram Equalisation

* As previously mentioned, image data may not spread across the whole bit-depth (`dtype`) of an image (array).
* The submodule `skimage.exposure` provides a range of functions for spreading an image's intensity over the full range.
* The simplest approach to this is to rescale the intensity levels.

In [ ]:
# Read a multidimensional TIF file, in this case a single channel with multiple z-slices.
myStack = io.imread('./assets/bbbc034v1/AICS_12_134_C=1.tif')

# Metadata for future use later
x_pixel_size = 65  # nm
y_pixel_size = 65  # nm
z_pixel_size = 290  # nm

# Take single slice
mySlice = myStack[26,:,:]

<div style="background-color:#abd9e9; border-radius: 5px; padding: 10pt"><strong>Task:</strong> Create a new cell below and use the <a href='https://scikit-image.org/docs/stable/api/skimage.exposure.html#skimage.exposure.rescale_intensity'><code>skimage.exposure.rescale_intensity()</code></a> function to rescale `mySlice` from 16-bit (assume it uses the full range) to 8-bit values. Check that the np array dtype is correct. Plot the two images side by side and their histograms beneath.</div>

In [ ]:
from skimage import exposure  # import the submodule first

myRescaledSlice = exposure.rescale_intensity(mySlice,in_range='uint16',out_range='uint8')  # rescale, as per the documentation and on-line examples

display(myRescaledSlice.dtype)  # will be uint16 because the output array is the same dtype as input (see documentation)

myRescaledSlice = myRescaledSlice.astype('uint8')  # convert dtype to uint8

display(myRescaledSlice.dtype)  # will be uint8 now

# Plot both images and their histograms
f, axes = plt.subplots(2,2)
(iO, iS, hO, hS) = axes.flatten()

iO.imshow(mySlice, cmap="gray", interpolation='none')
iO.grid(False)
iO.set_title("Original")

sns.distplot(mySlice.flatten(),kde=False,ax=hO)
hO.set_xlabel('Pixel Value')
hO.set_ylabel('Count')

iS.imshow(myRescaledSlice,  cmap="gray", interpolation='none')
iS.grid(False)
iS.set_title("Rescaled")

sns.distplot(myRescaledSlice.flatten(),kde=False,ax=hS)
hS.set_xlabel('Pixel Value')

plt.show()

<div style="background-color:#abd9e9; border-radius: 5px; padding: 10pt"><strong>Task:</strong> Now create a new cell below and map the data to the full 16-bit range. Check that the np array dtype is correct. Plot the two images side by side (use a full 16-bit colour mapping) and their histograms beneath.</div>

In [ ]:
myFullDRSlice = exposure.rescale_intensity(mySlice,in_range='image',out_range='uint16')  # now rescale using the image min and max (see documentation for in_range and 'image')

# Plot both images and their histograms
f, axes = plt.subplots(2,2)
(iO, iS, hO, hS) = axes.flatten()

iO.imshow(mySlice, cmap="gray", vmin=0, vmax=(2**16)-1, interpolation='none')
iO.grid(False)
iO.set_title("Original")

sns.distplot(mySlice.flatten(),bins=np.arange(0,(2**16)-1,2**8),kde=False,ax=hO)
hO.set_xlim([0,(2**16)-1])
hO.set_xlabel('Pixel Value')
hO.set_ylabel('Count')

iS.imshow(myFullDRSlice,  cmap="gray", vmin=0, vmax=(2**16)-1, interpolation='none')
iS.grid(False)
iS.set_title("Full Dynamic Range")

sns.distplot(myFullDRSlice.flatten(),bins=np.arange(0,(2**16)-1,2**8),kde=False,ax=hS)
hS.set_xlim([0,(2**16)-1])
hS.set_xlabel('Pixel Value')
hS.set_ylim([0,400000])


plt.show()

<div style="background-color:#abd9e9; border-radius: 5px; padding: 10pt"><strong>Task:</strong> Now create a new cell below and, using the codes above and the following tutorial, create a figure howing the original image, constrast stretched image, histogram equalised image and adaptive histogram equalised image, all with their histograms. You can find the tutorial at: <a href="https://scikit-image.org/docs/stable/auto_examples/color_exposure/plot_equalize.html#sphx-glr-auto-examples-color-exposure-plot-equalize-py">https://scikit-image.org/docs/stable/auto_examples/color_exposure/plot_equalize.html#sphx-glr-auto-examples-color-exposure-plot-equalize-py</a>.</div>

In [6]:
# Contrast stretching (stretch histogram so top and bottom 2% are saturated black/white)
p2, p98 = np.percentile(mySlice, (2, 98))
myCSSlice = exposure.rescale_intensity(mySlice, in_range=(p2, p98))

# [Global] Histogram Equalization
myEqualisedSlice = exposure.equalize_hist(mySlice)

# [Local] Adaptive Equalization, e.g. CLAHE
myAESlice = exposure.equalize_adapthist(mySlice, clip_limit=0.03)

# Display results and histograms
f, axes = plt.subplots(2,4)
(iO, iR, iE, iA, hO, hR, hE, hA) = axes.flatten()

iO.imshow(mySlice, cmap="gray", interpolation='none')
iO.set_axis_off()
iO.set_title("Original")

sns.distplot(mySlice.flatten(),kde=False,ax=hO)
hO.set_xlabel('Pixel Value')
hO.set_ylabel('Count')
hO.ticklabel_format(axis='y', style='scientific', scilimits=(0, 0))

iR.imshow(myCSSlice,  cmap="gray", interpolation='none')
iR.set_axis_off()
iR.set_title("Contrast Stretched")

sns.distplot(myCSSlice.flatten(),kde=False,ax=hR)
hR.set_xlabel('Pixel Value')
hR.ticklabel_format(axis='y', style='scientific', scilimits=(0, 0))

iE.imshow(myEqualisedSlice,  cmap="gray", interpolation='none')
iE.set_axis_off()
iE.set_title("Histogram Equalisation")

sns.distplot(myEqualisedSlice.flatten(),kde=False,ax=hE)
hE.set_xlabel('Pixel Value')
hE.ticklabel_format(axis='y', style='scientific', scilimits=(0, 0))

iA.imshow(myAESlice,  cmap="gray", interpolation='none')
iA.set_axis_off()
iA.set_title("Adaptive Equalisation")

sns.distplot(myAESlice.flatten(),kde=False,ax=hA)
hA.set_xlabel('Pixel Value')
hA.ticklabel_format(axis='y', style='scientific', scilimits=(0, 0))

plt.show()

## Image Filtering

* Many image processing tasks include filtering, either in the spatial or frequency domain.
* Again, `scitkit-image` has many of these filters built in to the submodule `scikit-image.filters`.

<div style="background-color:#abd9e9; border-radius: 5px; padding: 10pt"><strong>Task:</strong> Using the <a href="https://scikit-image.org/docs/stable/api/skimage.filters.html"><code>skimage.filters</code></a> submodule create a figure with a crop of the original slice (256-by-256 pixels, centred) and the results of applying a Gaussian blue; median filter; unsharp mask; sobel edge filter; and Meijering neuriteness ridge oeprator of the cropped region.</div>

In [7]:
from skimage import filters  # need to import filters submodule

# Crop a 256 by 256 grid centred in the image (no need to understand this line but it could be useful)
idx = tuple(np.meshgrid(np.arange(-128,128) + (mySlice.shape[0]//2),np.arange(-128,128) + (mySlice.shape[1]//2)))
myCrop = mySlice[idx]  # crop

# Gaussian Blur (with default kernel)
myGBSlice = filters.gaussian(myCrop)

# Median Filter (with default selem, i.e. Structuring ELEMent - similar to a kernel)
myMFSlice = filters.median(myCrop)

# Unsharp Mask with chosen radius (in pixels)
myUMSlice = filters.unsharp_mask(myCrop,radius=4.0)

# Sobel Edge Filter
mySESlice = filters.sobel(myCrop)

# Meijering Vesselness with user-chosen parameters
myMNSlice = filters.meijering(myCrop,sigmas=range(1,5),black_ridges=False)  # note: probably best to pad and unpad to remove edge artifacts

# Display results
f, axes = plt.subplots(2,3)
(iO, iG, iM, iU, iS, iF) = axes.flatten()

iO.imshow(myCrop, cmap="gray", interpolation='none')
iO.set_axis_off()
iO.set_title("Original")

iG.imshow(myGBSlice,  cmap="gray", interpolation='none')
iG.set_axis_off()
iG.set_title("Gaussian Blur")

iM.imshow(myMFSlice,  cmap="gray", interpolation='none')
iM.set_axis_off()
iM.set_title("Median Filter")

iU.imshow(myUMSlice,  cmap="gray", interpolation='none')
iU.set_axis_off()
iU.set_title("Unsharp Mask")

iS.imshow(mySESlice,  cmap="gray", interpolation='none')
iS.set_axis_off()
iS.set_title("Sobel Filter")

iF.imshow(myMNSlice,  cmap="gray", interpolation='none')
iF.set_axis_off()
iF.set_title("Meijering Neuriteness")

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Deconvolution

* One common operation in microscopy that takes place in the frequency domain is deconvolution.
* `scitkit-image.restoration` has a variety of denoising and deconvolution algorithms, including a Richardson-Lucy implementation.

In [8]:
import psf  # this is a new package that works out a mathematical PSF, but a measured PSF could also be used
from skimage import transform  # import this submodule to rescale the PSF (to account for anisotropy in z)

sz = 11
args = {
    'shape': (sz, sz),  # size of calculated psf array in pixels
    'dims': (x_pixel_size/1000*sz, y_pixel_size/1000*sz),  # size of array in microns
    'em_wavelen': 520.0,  # emission wavelength in nanometers
    'num_aperture': 1.25,  # numerical aperture
    'refr_index': 1.333,  # refractive index
    'magnification': 100,  # magnification
}

gauss = psf.PSF(psf.GAUSSIAN | psf.EMISSION, **args)

psf_ideal = gauss.volume()

# # Uncomment to display PSF before resizing for anisotropy
# f, axes = plt.subplots(2,2)
# (XZ, XY, null, ZY) = axes.flatten()
# f.suptitle("Gaussian PSF")

# ZY.imshow(psf_ideal[:,sz,:], cmap="gray", interpolation='none')
# ZY.grid(False)
# ZY.set_title("Central X-slice")

# XZ.imshow(psf_ideal[:,:,sz].T, cmap="gray", interpolation='none')
# XZ.grid(False)
# XZ.set_title("Central Y-slice")

# XY.imshow(psf_ideal[sz,:,:], cmap="gray", interpolation='none')
# XY.grid(False)
# XY.set_title("Central Z-slice")

# null.set_axis_off()  # clear unused subplot

# plt.tight_layout()
# plt.show()

# Resize for anisotropy of our image (this is a bit rough and can be done better - but it works for this example)
psf_rescaled = transform.resize(psf_ideal,
                                (np.ceil(psf_ideal.shape[0]*(x_pixel_size/z_pixel_size)),
                                 psf_ideal.shape[1],
                                 psf_ideal.shape[2]))
psf_rescaled = psf_rescaled/psf_rescaled.sum()

# Display PSF after resizing for anisotropy
f, axes = plt.subplots(2,2)
(XZ, XY, null, ZY) = axes.flatten()
f.suptitle("Gaussian PSF")

ZY.imshow(psf_rescaled[:,psf_rescaled.shape[1]//2+1,:], cmap="gray", interpolation='none')
ZY.grid(False)
ZY.set_title("Central X-slice")

XZ.imshow(psf_rescaled[:,:,psf_rescaled.shape[2]//2+1].T, cmap="gray", interpolation='none')
XZ.grid(False)
XZ.set_title("Central Y-slice")

XY.imshow(psf_rescaled[psf_rescaled.shape[0]//2+1,:,:], cmap="gray", interpolation='none')
XY.grid(False)
XY.set_title("Central Z-slice")

null.set_axis_off()  # clear unused subplot

plt.tight_layout()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<div style="background-color:#abd9e9; border-radius: 5px; padding: 10pt"><strong>Task:</strong> Using the <a href="https://scikit-image.org/docs/stable/api/skimage.restoration.html#skimage.restoration.richardson_lucy"><code>skimage.restoration.richardson_lucy</code></a> function, and in a new cell, deconvolve our 3D image for channel 1 (GFP) with the PSF defined above. Display a region of the central slice before and after convolution.</div>

In [9]:
import time  # so that we can time the deconvolution
from skimage import restoration  # import submodule that contains deconvolution algorithms

t = time.time()
# Restore Image using Richardson-Lucy algorithm
myFloatStack = myStack.astype('float64')/myStack.max()  # normalise and convert to float (needed for deconvolution with our PSF generated above)
deconvolved_RL = restoration.richardson_lucy(myStack.astype('float')/((2**16)-1), psf_rescaled, iterations=10)  # deconvolve

# Pick a small central region to plot
idx = np.meshgrid(myStack.shape[0]//2,np.arange(-128,128) + (myStack.shape[1]//2),np.arange(-128,128) + (myStack.shape[2]//2),indexing='ij')
idx = tuple([np.squeeze(idx[0]),np.squeeze(idx[1]),np.squeeze(idx[2])])

# Plot original and deconvolved image (just region defined above)
f, axes = plt.subplots(1,2)
(aO, aD) = axes.flatten()

aO.imshow(myStack[idx], cmap='gray')
aO.grid(False)
aO.set_title('Original Image')

aD.imshow(deconvolved_RL[idx], cmap='gray')
aD.grid(False)
aD.set_title('Richardson-Lucy Deconvolved')

plt.show()

print('Running time: {0} s'.format(time.time()-t))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Running time: 153.52439165115356 s


## Segmentation

* Here we must introduce the Python concept of Boolean or logical values: i.e. True and False
* True and False can be represented in arrays of `dtype` 'logical' or as arrays of 1s and 0s.
  * In both cases these are essentailly black and white images and can be displayed and processing as such
* There are two groups of thresholding algorithms available in `sciki-image`:
  1. Thresholding (found in `skimage.filters`), including Otsu and hysteresis thresholding
  2. More complex segmentation algorithms, e.g. active contours and the watershed algorithm (found in `skimage.segmentation`)

### Thresholding

* Usually we would combine thresholding with pre-processing, e.g. noise reduction or deconvolution, and post-processing, e.g. morphological operations to fill holes and smooth the resulting segmentation.

<div style="background-color:#abd9e9; border-radius: 5px; padding: 10pt"><strong>Task:</strong> Using the very helpful <a href="https://scikit-image.org/docs/stable/api/skimage.filters.html#skimage.filters.try_all_threshold"><code>skimage.filters.try_all_threshold</code></a> function see what a single slice of our nuclei-labelled channel looks like after different thresholding approaches.</div>

In [10]:
# Read a multidimensional TIF file, in this case a single channel with multiple z-slices.
myNucleiStack = io.imread('./assets/bbbc034v1/AICS_12_134_C=2.tif')
myNucleiSlice = myNucleiStack[myNucleiStack.shape[0]//2,:,:].astype('float64') # take central slice and make float

# Cropped area for viewing
idx = tuple(np.meshgrid(np.arange(-256,256) + (myNucleiSlice.shape[0]//2),np.arange(-256,256) + (myNucleiSlice.shape[0]//2),indexing='ij'))  # identify a central area for viewing
# idx = tuple([np.squeeze(idx[0]),np.squeeze(idx[1])])

# Normalise to use full range between 0.0 and 1.0 (allowing for the fact we have a couple of very bright spots in this place)
p2, p98 = np.percentile(myNucleiSlice, (2, 98))
myNucleiSlice = exposure.rescale_intensity(myNucleiSlice, in_range=(p2, p98), out_range=(0.0,1.0)).astype('float64')

# Use SciKit-Image's built-in function
f, axes = filters.try_all_threshold(myNucleiSlice[idx],verbose=False)

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<div style="background-color:#abd9e9; border-radius: 5px; padding: 10pt"><strong>Task:</strong> Pick the best segmentation by thresholding from your results and apply morphological (binary) closing, using <a href="https://scikit-image.org/docs/stable/api/skimage.morphology.html#skimage.morphology.binary_closing"><code>skimage.morphology.binary_closing</code></a>, to fill the small holes for a cleaner segmentation.</div>

In [11]:
from skimage import morphology  # import morphology submodule for closing

# Use thresholding based on Otsu's method
thresh = filters.threshold_otsu(myNucleiSlice)  # get threshold value
print('Thresholding at {0}/{1}'.format(thresh,myNucleiSlice.max()))
myBinary = myNucleiSlice > thresh  # make binary (segmented) image using threshold value

# Close (see https://imagej.net/MorphoLibJ#Morphological_filters if you're not comfortable with morphological operations)
myClosedBinary = morphology.binary_closing(myBinary,selem=morphology.disk(2))

# Plot
f, axes = plt.subplots(1,3)
(aO, aB, aC) = axes.flatten()

aO.imshow(myNucleiSlice[idx], cmap='gray')
aO.grid(False)
aO.set_title('Original Image')

aB.imshow(myBinary[idx], cmap='gray')  # Python can still apply colour maps to logical arrays
aB.grid(False)
aB.set_title('Binary Imaged')

aC.imshow(myClosedBinary[idx], cmap='gray')
aC.grid(False)
aC.set_title('After Closing')

plt.show()

Thresholding at 0.412109375/1.0


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Extracting Regions of Interest and Features

* Once segmented, we often want to measure a variety of features of our objects.

<div style="background-color:#abd9e9; border-radius: 5px; padding: 10pt"><strong>Task: </strong>In a new cell, use <a href="https://scikit-image.org/docs/stable/api/skimage.measure.html"><code>skimage.measure</code></a> to get the centroid, major and minor axis length, orientation, perimeter and intensity range for cells segmented in the previous task. Can you be sure all the detected objects are cells? Can you easily filter your results to only include those you trust?</div>

In [12]:
import pandas as pd  # see the prequisite course for more info on Pandas
from skimage import color  # used for creating an RGB label image
from skimage import measure
from skimage import segmentation

# create a labelled version of the binary image
myLabelledImage = measure.label(myClosedBinary)

# use regionprops_table and convert to pd.DataFrame as it's easier to filter
measurements = pd.DataFrame(measure.regionprops_table(myLabelledImage,
                                                      intensity_image=myNucleiSlice,
                                                      properties=('label',  # needed to keep track of which region is which
                                                                  'area',  # will use this filter based on size
                                                                  'centroid',
                                                                  'major_axis_length',
                                                                  'minor_axis_length',
                                                                  'max_intensity',
                                                                  'min_intensity',
                                                                  'mean_intensity',
                                                                  'orientation',
                                                                  'perimeter')))

print('Found {0} objects:'.format(measurements.shape[0]))

# Display a random 10 rows from the data frame to get an idea of our data
display(measurements.sample(10))

# Right, let's do some sensible filtering
query_string = 'area>=50 & minor_axis_length>0'  # create a query saying we want an area>20 pixels and minor axis>0 (i.e. not a line)
filtered_measurements = measurements.query(query_string)  # apply the query

print('Filtered to keep {0} objects:'.format(filtered_measurements.shape[0]))
display(filtered_measurements.describe())  # show summary statistics of filtered blobs

# Let's replot the labels after filtering
removed_objects = measurements[~measurements['label'].isin(filtered_measurements['label'])]['label'].dropna().values  # get list of rows removed
filteredLabelledImage = myLabelledImage.copy()  # create a copy of the labelled image
for ido in removed_objects:  # for each removed row
    filteredLabelledImage[filteredLabelledImage==ido] = 0  # set pixels to zero
(relabelledClosedBinary, null, null) = segmentation.relabel_sequential(filteredLabelledImage)

f, axes = plt.subplots(1,2)
(axO, axF) = axes.flatten()

axO.imshow(color.label2rgb(myLabelledImage,image=myNucleiSlice))
axO.grid(False)
axO.set_title('All Objects')

axF.imshow(color.label2rgb(relabelledClosedBinary,image=myNucleiSlice))
axF.grid(False)
axF.set_title('Filtered Objects')

plt.show()

Found 3658 objects:


,label,area,centroid-0,centroid-1,major_axis_length,minor_axis_length,max_intensity,min_intensity,mean_intensity,orientation,perimeter
3093,3094,1,871,581,0.0,0.0,0.450980,0.450980,0.450980,0.785398,0.0
2980,2981,1,851,399,0.0,0.0,0.450980,0.450980,0.450980,0.785398,0.0
3519,3520,1,990,249,0.0,0.0,0.431373,0.431373,0.431373,0.785398,0.0
197,198,1,41,0,0.0,0.0,0.529412,0.529412,0.529412,0.785398,0.0
2299,2300,1,648,474,0.0,0.0,0.509804,0.509804,0.509804,0.785398,0.0
862,863,1,242,238,0.0,0.0,0.431373,0.431373,0.431373,0.785398,0.0
1351,1352,1,372,642,0.0,0.0,0.450980,0.450980,0.450980,0.785398,0.0
766,767,1,220,215,0.0,0.0,0.509804,0.509804,0.509804,0.785398,0.0
2572,2573,1,720,890,0.0,0.0,0.431373,0.431373,0.431373,0.785398,0.0
2710,2711,1,789,355,0.0,0.0,0.431373,0.431373,0.431373,0.785398,0.0


Filtered to keep 31 objects:


,label,area,centroid-0,centroid-1,major_axis_length,minor_axis_length,max_intensity,min_intensity,mean_intensity,orientation,perimeter
count,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000
mean,1878.645161,17713.290323,591.225806,524.096774,162.609083,95.520215,0.964579,0.017078,0.499883,-0.093176,969.391086
std,1268.760538,20659.131500,343.882898,367.131979,139.975964,81.105569,0.100035,0.029841,0.099560,1.041356,1068.588055
min,1.000000,56.000000,22.000000,2.000000,10.190223,7.092109,0.529412,0.000000,0.320261,-1.569815,29.627417
25%,742.000000,174.500000,266.500000,139.000000,23.879182,14.750254,1.000000,0.000000,0.421202,-0.956535,86.760931
50%,2103.000000,10285.000000,646.000000,576.000000,178.266816,80.131543,1.000000,0.000000,0.498568,-0.131744,751.299603
75%,3033.500000,29558.500000,864.000000,884.000000,224.900476,172.788150,1.000000,0.029412,0.544277,0.713341,1268.141882
max,3631.000000,65965.000000,1020.000000,1016.000000,464.994150,214.204893,1.000000,0.117647,0.722630,1.547205,4012.962225


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Summary

* Appreciate the capabilities of `scikit-image` for image processing in a Python environment
* Apply known image processing techniques (e.g. smoothing) in a Python environment
* Recognise additional image processing techniques (e.g. deconvolution) that are possible in a Python environment
* Relate global grayscale thresholding and the logical array to segmentation and binary images
* Extract features of objects from segmented images